In [13]:
import pandas as pd
from google.cloud import storage
import io
import json

In [14]:
# Set up the GCS client
client = storage.Client()

In [15]:
# Set the GCS bucket and file names
bucket_name = 'de-zoomcamp-2023-project-datalake-bucket_bright-aloe-381618'
# blob_name = 'data/ingest_germany_20230406.json'
blob_name = 'data/ingest_districts_20230407.json'

In [16]:
import re

input_str = blob_name
print(input_str)
match = re.search(r'_([^_]+)_', input_str)
result = match.group(1)

print(result) # "germany"


data/ingest_districts_20230407.json
districts


In [17]:
# Get a handle to the GCS bucket and file
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

In [18]:
# Download the file contents to a BytesIO object
bytes_io = io.BytesIO()
blob.download_to_file(bytes_io)
# Reset the BytesIO object to the beginning of the stream
bytes_io.seek(0)
# Open the BytesIO object in binary mode
binary_io = io.BufferedReader(bytes_io)
# Load the contents of the file into a Pandas DataFrame
df = pd.json_normalize(json.loads(bytes_io.read().decode()))

In [19]:
print(df.head())

  data.10041.ags              data.10041.name            data.10041.county  \
0          10041  Regionalverband Saarbrücken  LK Stadtverband Saarbrücken   

  data.10041.state  data.10041.population  data.10041.cases  \
0         Saarland                 327284            154016   

   data.10041.deaths  data.10041.casesPerWeek  data.10041.deathsPerWeek  \
0                983                       93                         3   

  data.10041.stateAbbreviation  ...  data.09780.casesPer100k  \
0                           SL  ...             49924.937342   

   data.09780.delta.cases  data.09780.delta.deaths  \
0                       0                        0   

   data.09780.delta.recovered  data.09780.delta.weekIncidence  \
0                          22                       -3.180621   

            meta.source                         meta.contact  \
0  Robert Koch-Institut  Marlon Lueckert (m.lueckert@me.com)   

                                    meta.info           meta.lastUp

In [7]:
# df = df.rename(columns={
#     'cases': 'cases',
#     'deaths': 'deaths',
#     'recovered': 'recovered',
#     'weekIncidence': 'weekIncidence',
#     'casesPer100k': 'casesPer100k',
#     'casesPerWeek': 'casesPerWeek',
#     'deathsPerWeek': 'deathsPerWeek',
#     'delta.cases': 'delta_cases',
#     'delta.deaths': 'delta_deaths',
#     'delta.recovered': 'delta_recovered',
#     'delta.weekIncidence': 'delta_weekIncidence',
#     'r.value': 'r_value',
#     'r.rValue4Days.value': 'r_rValue4Days_value',
#     'r.rValue4Days.date': 'r_rValue4Days_date',
#     'r.rValue7Days.value': 'r_rValue7Days_value',
#     'r.rValue7Days.date': 'r_rValue7Days_date',
#     'r.lastUpdate': 'r_lastUpdate',
#     'hospitalization.cases7Days': 'hospitalization_cases7Days',
#     'hospitalization.incidence7Days': 'hospitalization_incidence7Days',
#     'hospitalization.date': 'hospitalization_date',
#     'hospitalization.lastUpdate': 'hospitalization_lastUpdate',
#     'meta.source': 'meta_source',
#     'meta.contact': 'meta_contact',
#     'meta.info': 'meta_info',
#     'meta.lastUpdate': 'meta_lastUpdate',
#     'meta.lastCheckedForUpdate': 'meta_lastCheckedForUpdate'
# })

# create a mapping dictionary for renaming the columns
mapping = {col: col.replace('.', '_') for col in df.columns if '.' in col}

# use the mapping dictionary to rename the columns
df = df.rename(columns=mapping)

In [137]:
print(df.dtypes)

cases                               int64
deaths                              int64
recovered                           int64
weekIncidence                     float64
casesPer100k                      float64
casesPerWeek                        int64
deathsPerWeek                       int64
delta_cases                         int64
delta_deaths                        int64
delta_recovered                     int64
delta_weekIncidence               float64
r_value                           float64
r_rValue4Days_value               float64
r_rValue4Days_date                 object
r_rValue7Days_value               float64
r_rValue7Days_date                 object
r_lastUpdate                       object
hospitalization_cases7Days          int64
hospitalization_incidence7Days    float64
hospitalization_date               object
hospitalization_lastUpdate         object
meta_source                        object
meta_contact                       object
meta_info                         

In [148]:
# Set the BigQuery dataset and table names
project_id = 'bright-aloe-381618'
dataset_id = 'de_zoomcamp_2023_project_dataset'
table_id = 'covid_figures_germany'
destination_table = 'bright-aloe-381618.de_zoomcamp_2023_project_dataset.covid_figures_germany'
# Load the Pandas DataFrame to a BigQuery table
df.to_gbq(destination_table=f"{project_id}.{table_id}",
             project_id=project_id,
             if_exists='append')